# Project1 (Veronica) experiments: E1 / E2 / E3

This notebook assumes you have already trained the baseline policies (10–13 notebooks) and saved best weights in `../artifacts/runs/...` with `../artifacts/selected_runs.json`.

- **E1**: stochastic flex-price $r^*_s(k)$ vs volatility multiplier $k$ in bad regime
- **E2**: policy-recipe test (no retraining): run trained Taylor & ModTaylor policies under modified uncertainty and compare moments + CRN overlay
- **E3**: optional retraining under modified uncertainty (expensive)


In [ ]:

import os, sys, json, time
import numpy as np
import torch
import matplotlib.pyplot as plt

sys.path.append('..')

from src.config import ModelParams, TrainConfig
from src.deqn import PolicyNetwork, Trainer
from src.model_common import unpack_state
from src.io_utils import load_json, load_torch, ensure_dir
from src.flex_stochastic import compute_rstar_by_regime

ARTIFACTS_ROOT = os.path.abspath('../artifacts')
RESULTS_DIR = os.path.join(ARTIFACTS_ROOT, 'results')
os.makedirs(RESULTS_DIR, exist_ok=True)

# --- experiment grid ---
k_grid = [1.0, 1.25, 1.5, 2.0, 3.0]

# --- simulation lengths (tune if needed) ---
T_rstar, burn_rstar = 120_000, 20_000
T_policy, burn_policy = 40_000, 5_000
T_overlay = 400
T_dist, burn_dist = 60_000, 5_000

SEED = 123
np.random.seed(SEED)
torch.manual_seed(SEED)


## Helper: load trained policy networks

In [ ]:

from pathlib import Path

def _load_selected_runs(path=os.path.join(ARTIFACTS_ROOT,'selected_runs.json')):
    with open(path,'r',encoding='utf-8') as f:
        return json.load(f)

sel = _load_selected_runs()
sel


In [ ]:

# Load Taylor and ModTaylor best weights

def load_trainer_for_policy(policy_name: str, params: ModelParams):
    run_dir = sel[policy_name]['run_dir']
    weights_path = os.path.join(run_dir, sel[policy_name].get('best_weights_name','weights_best.pt'))

    # Network dims
    if policy_name in ('taylor','mod_taylor'):
        d_in, d_out = 5, 8
    elif policy_name=='discretion':
        d_in, d_out = 5, 11
    elif policy_name=='commitment':
        d_in, d_out = 7, 13
    else:
        raise ValueError(policy_name)

    # Use a light cfg for inference
    cfg = TrainConfig(batch_size=256, steps=1, lr=1e-5, gh_n_train=3, artifacts_root=ARTIFACTS_ROOT,
                      cpu_num_threads=4, cpu_num_interop_threads=1)

    net = PolicyNetwork(d_in=d_in, d_out=d_out, hidden=cfg.hidden_layers, activation=cfg.activation)
    sd = load_torch(weights_path, map_location=params.device)
    net.load_state_dict(sd)
    net.eval()

    trainer = Trainer(params=params, cfg=cfg, policy=policy_name, net=net,
                      rbar_by_regime=None, gh_n=3)
    return trainer


## E1 — Stochastic flex-price $r^*_s(k)$ sweep

In [ ]:

E1_rows = []
for k in k_grid:
    t0 = time.time()
    p = ModelParams(device='cpu', dtype=torch.float32,
                    sigma_A_bad_mult=k, sigma_g_bad_mult=k, sigma_tau_bad_mult=k)
    res = compute_rstar_by_regime(p, T=T_rstar, burn_in=burn_rstar, seed=SEED)
    E1_rows.append({
        'k': k,
        'rstar_s0': res.rstar_by_regime[0],
        'rstar_s1': res.rstar_by_regime[1],
        'delta_rstar': res.delta_rstar,
        'n0': res.samples_by_regime[0],
        'n1': res.samples_by_regime[1],
        'seconds': time.time()-t0,
    })

import pandas as pd
E1 = pd.DataFrame(E1_rows)
E1_path = os.path.join(RESULTS_DIR, 'project1_E1_rstar_sweep.csv')
E1.to_csv(E1_path, index=False)
E1


In [ ]:

plt.figure()
plt.plot(E1['k'], E1['delta_rstar'], marker='o')
plt.xlabel('k (bad-regime volatility multiplier)')
plt.ylabel('Δ r* (s=1 minus s=0)')
plt.title('E1: stochastic flex-price Δr*(k)')
plt.grid(True)
plt.show()


## E2 — Policy recipe test (no retraining)

In [ ]:

# Load baseline trainers once (they will be re-used; only ModelParams changes per k)
base_params = ModelParams(device='cpu', dtype=torch.float32)
tr_taylor = load_trainer_for_policy('taylor', base_params)
tr_mod    = load_trainer_for_policy('mod_taylor', base_params)

@torch.no_grad()
def simulate_policy_moments(trainer: Trainer, params: ModelParams, T: int, burn_in: int, seed: int):
    """Simulate policy under params (including sigma multipliers) and return time series."""
    torch.manual_seed(seed)
    np.random.seed(seed)

    # swap params inside trainer (keeps net)
    trainer.params = params.to_torch()
    trainer._eps_grid, trainer._w_grid = None, None  # will be cached on first expectation call
    trainer._eps_grid, trainer._w_grid = trainer._eps_grid, trainer._w_grid

    # init
    x = trainer.simulate_initial_state(1, commitment_sss=None).to(params.device)

    series = {'pi':[], 'i':[], 'y':[], 'c':[], 's':[]}
    for t in range(T):
        out = trainer._policy_outputs(x)
        ids = trainer._identities(x, out)
        # implied nominal rate for taylor policies is deterministic rule inside residuals; use helper
        if trainer.policy=='taylor':
            from src.policy_rules import i_taylor
            it = i_taylor(params, ids, out)
        elif trainer.policy=='mod_taylor':
            from src.policy_rules import i_modified_taylor
            it = i_modified_taylor(params, ids, out)
        else:
            it = torch.full_like(out['pi'], float('nan'))

        series['pi'].append(float(out['pi'].cpu()))
        series['i'].append(float(it.cpu()))
        series['y'].append(float(ids['y'].cpu()))
        series['c'].append(float(out['c'].cpu()))
        series['s'].append(int(unpack_state(x, trainer.policy).s.item()))

        x = trainer._step_state(x)

    # burn
    for k in series:
        series[k] = np.asarray(series[k])[burn_in:]
    return series


def moments_by_regime(series):
    out = {}
    s = series['s']
    for reg in (0,1):
        m = (s==reg)
        out[reg] = {
            'n': int(m.sum()),
            'pi_mean': float(series['pi'][m].mean()) if m.sum()>0 else np.nan,
            'pi_std': float(series['pi'][m].std()) if m.sum()>0 else np.nan,
            'i_mean': float(series['i'][m].mean()) if m.sum()>0 else np.nan,
            'i_std': float(series['i'][m].std()) if m.sum()>0 else np.nan,
            'y_mean': float(series['y'][m].mean()) if m.sum()>0 else np.nan,
            'y_std': float(series['y'][m].std()) if m.sum()>0 else np.nan,
            'c_mean': float(series['c'][m].mean()) if m.sum()>0 else np.nan,
            'c_std': float(series['c'][m].std()) if m.sum()>0 else np.nan,
        }
    return out

E2_rows=[]
for k in k_grid:
    p = ModelParams(device='cpu', dtype=torch.float32,
                    sigma_A_bad_mult=k, sigma_g_bad_mult=k, sigma_tau_bad_mult=k)

    s_t = simulate_policy_moments(tr_taylor, p, T_policy, burn_policy, SEED)
    s_m = simulate_policy_moments(tr_mod,   p, T_policy, burn_policy, SEED)

    mt = moments_by_regime(s_t)
    mm = moments_by_regime(s_m)

    for reg in (0,1):
        E2_rows.append({'policy':'taylor','k':k,'regime':reg, **mt[reg]})
        E2_rows.append({'policy':'mod_taylor','k':k,'regime':reg, **mm[reg]})

E2 = pd.DataFrame(E2_rows)
E2_path = os.path.join(RESULTS_DIR, 'project1_E2_policy_recipe.csv')
E2.to_csv(E2_path, index=False)
E2.head(10)


### E2 plot: mean inflation by regime vs k

In [ ]:

for policy in ['taylor','mod_taylor']:
    plt.figure()
    for reg in [0,1]:
        d = E2[(E2.policy==policy) & (E2.regime==reg)].sort_values('k')
        plt.plot(d['k'], d['pi_mean'], marker='o', label=f's={reg}')
    plt.xlabel('k')
    plt.ylabel('mean(pi)')
    plt.title(f'E2: mean inflation vs k ({policy})')
    plt.grid(True)
    plt.legend()
    plt.show()


## E2 visual — Common-random-numbers overlay (k=1 vs k=2)

In [ ]:

@torch.no_grad()
def simulate_overlay(trainer: Trainer, params: ModelParams, T: int, seed: int):
    torch.manual_seed(seed)
    np.random.seed(seed)
    trainer.params = params.to_torch()
    x = trainer.simulate_initial_state(1, commitment_sss=None).to(params.device)

    out_series = {'pi':[], 'i':[], 'y':[], 'c':[], 's':[]}
    for t in range(T):
        out = trainer._policy_outputs(x)
        ids = trainer._identities(x, out)
        if trainer.policy=='taylor':
            from src.policy_rules import i_taylor
            it = i_taylor(params, ids, out)
        else:
            from src.policy_rules import i_modified_taylor
            it = i_modified_taylor(params, ids, out)
        out_series['pi'].append(float(out['pi'].cpu()))
        out_series['i'].append(float(it.cpu()))
        out_series['y'].append(float(ids['y'].cpu()))
        out_series['c'].append(float(out['c'].cpu()))
        out_series['s'].append(int(unpack_state(x, trainer.policy).s.item()))
        x = trainer._step_state(x)
    for k in out_series:
        out_series[k] = np.asarray(out_series[k])
    return out_series

p1 = ModelParams(device='cpu', dtype=torch.float32,
                 sigma_A_bad_mult=1.0, sigma_g_bad_mult=1.0, sigma_tau_bad_mult=1.0)
p2 = ModelParams(device='cpu', dtype=torch.float32,
                 sigma_A_bad_mult=2.0, sigma_g_bad_mult=2.0, sigma_tau_bad_mult=2.0)

ov = {
    ('taylor',1.0): simulate_overlay(tr_taylor, p1, T_overlay, SEED),
    ('taylor',2.0): simulate_overlay(tr_taylor, p2, T_overlay, SEED),
    ('mod_taylor',1.0): simulate_overlay(tr_mod, p1, T_overlay, SEED),
    ('mod_taylor',2.0): simulate_overlay(tr_mod, p2, T_overlay, SEED),
}

for var in ['pi','i','y','c']:
    plt.figure()
    for key, ls in [(( 'taylor',1.0),'-'),(( 'taylor',2.0),'--'),(( 'mod_taylor',1.0),'-.'),(( 'mod_taylor',2.0),':')]:
        plt.plot(ov[key][var], linestyle=ls, label=f'{key[0]} k={key[1]}')
    plt.xlabel('t')
    plt.ylabel(var)
    plt.title(f'CRN overlay: {var} (same seed path)')
    plt.grid(True)
    plt.legend()
    plt.show()


## E2 distributions — histograms + tail quantiles (k=1 vs k=2)

In [ ]:

# Recompute longer series only for k=1 vs k=2
p1 = ModelParams(device='cpu', dtype=torch.float32,
                 sigma_A_bad_mult=1.0, sigma_g_bad_mult=1.0, sigma_tau_bad_mult=1.0)
p2 = ModelParams(device='cpu', dtype=torch.float32,
                 sigma_A_bad_mult=2.0, sigma_g_bad_mult=2.0, sigma_tau_bad_mult=2.0)

s_t1 = simulate_policy_moments(tr_taylor, p1, T_dist, burn_dist, SEED)
s_t2 = simulate_policy_moments(tr_taylor, p2, T_dist, burn_dist, SEED)
s_m1 = simulate_policy_moments(tr_mod,   p1, T_dist, burn_dist, SEED)
s_m2 = simulate_policy_moments(tr_mod,   p2, T_dist, burn_dist, SEED)


def quantiles(series, reg, q=(0.05,0.5,0.95)):
    m = (series['s']==reg)
    return {f'q{int(100*qq)}': float(np.quantile(series['pi'][m], qq)) for qq in q}

rows=[]
for policy, a, b in [('taylor',s_t1,s_t2),('mod_taylor',s_m1,s_m2)]:
    for reg in (0,1):
        rows.append({'policy':policy,'regime':reg,'k':1.0,'n':int((a['s']==reg).sum()), **quantiles(a,reg)})
        rows.append({'policy':policy,'regime':reg,'k':2.0,'n':int((b['s']==reg).sum()), **quantiles(b,reg)})

Q = pd.DataFrame(rows)
Q_path = os.path.join(RESULTS_DIR, 'project1_E2_quantiles_k1_vs_k2.csv')
Q.to_csv(Q_path, index=False)
Q


In [ ]:

# Histograms for inflation by regime

def plot_hist(policy_name, s_k1, s_k2, reg):
    plt.figure()
    m1 = (s_k1['s']==reg)
    m2 = (s_k2['s']==reg)
    plt.hist(s_k1['pi'][m1], bins=60, alpha=0.5, density=True, label='k=1')
    plt.hist(s_k2['pi'][m2], bins=60, alpha=0.5, density=True, label='k=2')
    plt.title(f'{policy_name}: inflation distribution in regime s={reg}')
    plt.xlabel('pi')
    plt.ylabel('density')
    plt.legend()
    plt.grid(True)
    plt.show()

for reg in (0,1):
    plot_hist('taylor', s_t1, s_t2, reg)
    plot_hist('mod_taylor', s_m1, s_m2, reg)


## E3 — Optional retraining (expensive)

## Sanity checks: Markov regime probabilities and stationarity
These checks confirm that the Markov transition matrix `P[s_next, s]` is normalized and that
the simulated regime frequencies match the stationary distribution implied by `P`.


In [ ]:

# --- Markov regime sanity checks ---
P = base_params.P.detach().cpu().numpy()  # P[s_next, s]
print("P (rows = s_next, cols = s):\n", P)
print("Column sums (should be 1):", P.sum(axis=0))
print("Min/Max prob:", P.min(), P.max())

assert np.allclose(P.sum(axis=0), 1.0, atol=1e-10), "Transition probs do not sum to 1 by column."
assert (P >= -1e-14).all() and (P <= 1+1e-14).all(), "Transition probs outside [0,1]."

# Stationary distribution: solve pi = P * pi (with column-stochastic convention)
eigvals, eigvecs = np.linalg.eig(P)
idx = np.argmin(np.abs(eigvals - 1.0))
pi_stat = np.real(eigvecs[:, idx])
pi_stat = pi_stat / pi_stat.sum()
pi_stat = np.clip(pi_stat, 0, None)
pi_stat = pi_stat / pi_stat.sum()
print("Stationary distribution implied by P:", pi_stat)

# Empirical frequencies from a long Markov simulation (no economics, just regimes)
def simulate_markov_only(P, T=200000, s0=0, seed=123):
    rng = np.random.default_rng(seed)
    s = np.empty(T, dtype=np.int64)
    s[0] = s0
    for t in range(1, T):
        u = rng.random()
        # prob of next=0 is P[0, current]
        s[t] = 0 if u < P[0, s[t-1]] else 1
    return s

s_path = simulate_markov_only(P, T=300000, s0=0, seed=123)
emp = np.bincount(s_path, minlength=2) / len(s_path)
print("Empirical regime frequencies:", emp)

# Quick plot: stationary vs empirical
df = pd.DataFrame({
    "prob": [pi_stat[0], pi_stat[1], emp[0], emp[1]],
    "type": ["stationary","stationary","empirical","empirical"],
    "state": [0,1,0,1],
})
plt.figure()
for ttype in ["stationary","empirical"]:
    sub = df[df["type"]==ttype].sort_values("state")
    plt.plot(sub["state"], sub["prob"], marker='o', label=ttype)
plt.xticks([0,1], ["normal (0)", "bad (1)"])
plt.ylabel("probability")
plt.title("Markov regimes: stationary vs empirical")
plt.legend()
plt.show()

# (Optional) verify the expectation weights used in the code match P[:, s_t]
for s_t in [0,1]:
    w = P[:, s_t]
    print(f"Expectation regime weights when s_t={s_t}: P[:,{s_t}] =", w, " sum=", w.sum())


In [ ]:

RUN_E3 = False
E3_k_grid = [1.25, 2.0]

if RUN_E3:
    # Minimal retraining loop for Taylor + ModTaylor under each k.
    # Saves new runs under artifacts/runs and writes artifacts/results/project1_E3_runs.json
    from src.io_utils import make_run_dir, save_run_metadata, save_selected_run, save_torch
    from src.steady_states import solve_flexprice_sss, export_rbar_tensor

    out = {}
    for policy in ['taylor','mod_taylor']:
        out[policy] = {}
        for k in E3_k_grid:
            params = ModelParams(device='cpu', dtype=torch.float32,
                                sigma_A_bad_mult=k, sigma_g_bad_mult=k, sigma_tau_bad_mult=k)

            run_dir = make_run_dir('../artifacts', policy=policy, tag=f'E3_k{k}', seed=SEED)

            # fast but reasonable plan (same as train notebooks)
            cfgA = TrainConfig(seed=SEED, steps=6000 if policy in ('taylor','mod_taylor') else 8000,
                               batch_size=128 if policy in ('taylor','mod_taylor') else 64,
                               lr=2e-5, gh_n_train=2, artifacts_root='../artifacts', run_dir=run_dir,
                               save_best=True, cpu_num_threads=4, cpu_num_interop_threads=1,
                               replay_buffer_size=120_000, val_size=4096, val_every=400,
                               reduce_lr_on_plateau=False, early_stopping=False)
            cfgB = TrainConfig(seed=SEED, steps=4000 if policy in ('taylor','mod_taylor') else 6000,
                               batch_size=96 if policy in ('taylor','mod_taylor') else 48,
                               lr=1e-5, gh_n_train=3, artifacts_root='../artifacts', run_dir=run_dir,
                               save_best=True, cpu_num_threads=4, cpu_num_interop_threads=1,
                               replay_buffer_size=120_000, val_size=4096, val_every=400,
                               reduce_lr_on_plateau=False, early_stopping=False)

            # rbar only matters for mod_taylor (rule uses r*_s)
            flex = solve_flexprice_sss(params)
            rbar = export_rbar_tensor(params, flex)

            net = PolicyNetwork(d_in=5, d_out=8, hidden=cfgA.hidden_layers, activation=cfgA.activation)
            trainer = Trainer(params=params, cfg=cfgA, policy=policy, net=net,
                              rbar_by_regime=rbar if policy=='mod_taylor' else None,
                              gh_n=cfgA.gh_n_train)

            # init population
            x0 = trainer.simulate_initial_state(1024)

            trainer.train(x0, steps=cfgA.steps)
            trainer.cfg = cfgB
            trainer.gh_n = cfgB.gh_n_train
            trainer.train(x0, steps=cfgB.steps)

            # save weights
            save_torch(os.path.join(run_dir,'weights_final.pt'), net.state_dict())

            out[policy][str(k)] = {'run_dir': run_dir, 'weights': 'weights_final.pt'}

    path = os.path.join(RESULTS_DIR,'project1_E3_runs.json')
    with open(path,'w',encoding='utf-8') as f:
        json.dump(out, f, indent=2)
    print('saved', path)
